In [160]:
import talib
import math
from functools import reduce
import numpy as np

In [31]:
import pandas as pd

def mystrategy(data):
    
    股本 = data.get('股本合計', 1)
    price = data.get('收盤價', 120)
    當天股價 = price[:股本.index[-1]].iloc[-1]
    當天股本 = 股本.iloc[-1]
    市值 = 當天股本 * 當天股價 / 10 * 1000
   

    df1 = toSeasonal(data.get('投資活動之淨現金流入（流出）', 5))
    df2 = toSeasonal(data.get('營業活動之淨現金流入（流出）', 5))
    自由現金流 = (df1 + df2).iloc[-4:].mean()
    
    
    稅後淨利 = data.get('本期淨利（淨損）', 1)
    
    # 股東權益，有兩個名稱，有些公司叫做權益總計，有些叫做權益總額
    # 所以得把它們抓出來
    權益總計 = data.get('權益總計', 1)
    權益總額 = data.get('權益總額', 1)
    
    # 並且把它們合併起來
    權益總計.fillna(權益總額, inplace=True)
        
    股東權益報酬率 = 稅後淨利.iloc[-1] / 權益總計.iloc[-1]
    
    
    營業利益 = data.get('營業利益（損失）', 5)
    營業利益成長率 = (營業利益.iloc[-1] / 營業利益.iloc[-5] - 1) * 100
    
    
    當月營收 = data.get('當月營收', 4) * 1000
    當季營收 = 當月營收.iloc[-4:].sum()
    市值營收比 = 市值 / 當季營收
    
    
    condition1 = (市值 < 1e10)
    condition2 = 自由現金流 > 0
    condition3 = 股東權益報酬率 > 0
    condition4 = 營業利益成長率 > 0
    condition5 = 市值營收比 < 5
    
    select_stock = condition1 & condition2 & condition3 & condition4 & condition5
    
    return select_stock[select_stock]

In [32]:
import pandas as pd

def toSeasonal(df):
    season4 = df[df.index.month == 3]
    season1 = df[df.index.month == 5]
    season2 = df[df.index.month == 8]
    season3 = df[df.index.month == 11]

    season1.index = season1.index.year
    season2.index = season2.index.year
    season3.index = season3.index.year
    season4.index = season4.index.year - 1

    newseason1 = season1
    newseason2 = season2 - season1.reindex_like(season2)
    newseason3 = season3 - season2.reindex_like(season3)
    newseason4 = season4 - season3.reindex_like(season4)

    newseason1.index = pd.to_datetime(newseason1.index.astype(str) + '-05-15')
    newseason2.index = pd.to_datetime(newseason2.index.astype(str) + '-08-14')
    newseason3.index = pd.to_datetime(newseason3.index.astype(str) + '-11-14')
    newseason4.index = pd.to_datetime((newseason4.index + 1).astype(str) + '-03-31')

    return newseason1.append(newseason2).append(newseason3).append(newseason4).sort_index()

def turning(data):
    # capital = data.get('股本合計',1)
# price = data.get('收盤價',100)
# # 先拿出計算市值需要用到的財務數據

    股本 = data.get('股本合計', 1)
    price = data.get('收盤價', 100)
    %matplotlib inline
    當月營收 = data.get('當月營收', 4) * 1000
    當季營收 = 當月營收.iloc[-4:].sum() # 取近4個月營收總和，當作一季的月營收（4也可以改變）

    # 將每季累計的財務數據，轉換成單季


    # 計算自由現金流 = 營業活動之淨現金流入 - 投資活動之淨現金流出
    投資現金流 = toSeasonal(data.get('投資活動之淨現金流入（流出）', 5))
    營業現金流 = toSeasonal(data.get('營業活動之淨現金流入（流出）', 5))
    自由現金流 = (投資現金流 + 營業現金流).iloc[-4:].sum()

    稅後淨利 = data.get('本期淨利（淨損）', 5)
    給自由現金的稅後淨利 = data.get('本期淨利（淨損）', 4)
    # 修正：因為有些股東權益的名稱叫作「權益總計」有些叫作「權益總額」，所以要先將這兩個dataframe合併起來喔！
    權益總計 = data.get('權益總計', 5)
    權益總額 = data.get('權益總額', 5)

    # 把它們合併起來（將「權益總計」為NaN的部分填上「權益總額」）
    權益總計.fillna(權益總額, inplace=True)

    股東權益報酬率 = 稅後淨利.iloc[-1] / 權益總計.iloc[-1]
    股東權益報酬率.describe()

    上季股東權益報酬率 = 稅後淨利.iloc[-5] / 權益總計.iloc[-5]
    上季股東權益報酬率.describe()

    營業利益 = data.get('營業利益（損失）', 5)
    營業利益成長率 = (營業利益.iloc[-1] / 營業利益.iloc[-5] - 1) * 100
    營業利益成長率.describe()

    # current
    流動資產 = data.get('流動資產合計',5)
    流動負債 = data.get('流動負債合計',5)
    current_ratio = 流動資產/流動負債
    # liability
    long_liability = data.get('非流動負債合計',5)
    
    #銷貨
    銷貨收入 = data.get('銷貨收入',6)
    #capital
    stocks = data.get('普通股股本',5)
    #inventory
    control_season = 2
    inventory = data.get('存貨合計',6)
    asset = data.get('資產總計',6)
    no_turnover = inventory/asset
    turnover_ratio = 1 - no_turnover
    turnover_growth_rate = (turnover_ratio.iloc[-control_season] /turnover_ratio.iloc[-control_season-4])
    turnover_growth_rate[0]
    
    資產週轉率 = 銷貨收入/asset
    
    #淨值比
    流通股數= stocks / 10
    每股淨值= 權益總計 / 流通股數
    股價淨值比 = price.iloc[-1]/每股淨值.iloc[-1]  
    
    營業收入 = data.get('營業收入合計',5)
    
    c1 = 當季營收 > 0
    #c2 = 自由現金流 >0
    c3 = (營業現金流.iloc[-1]>0)
    c4 = long_liability.iloc[-1] < long_liability.iloc[-5]
    c5 = current_ratio.iloc[-1] >current_ratio.iloc[-5]
    c6 = (stocks.iloc[-1] - stocks.iloc[-5]) <= 0
    c7 = 股東權益報酬率 > 上季股東權益報酬率
    c8 = 營業利益成長率>0 #把毛益率改成營業利益率
    c9 = turnover_growth_rate >0
    c10 = 股價淨值比 > 1
    c11 = 稅後淨利.iloc[-1] > 0

        
    # 將條件做交集（&）
    select_stock = c1& c2 & c3& c4& c5& c6& c7& c8& c9 & c10 & c11

    # 選出的股票
    select_stock = select_stock[select_stock]
    return select_stock

## Yello func

In [203]:
from finlab.backtest import backtest
from finlab.data import Data
import datetime
%matplotlib inline
data = Data()
營業收入 = data.get('營業收入合計',5)

    # capital = data.get('股本合計',1)
# price = data.get('收盤價',100)
# # 先拿出計算市值需要用到的財務數據

股本 = data.get('股本合計', 1)
price = data.get('收盤價', 100)
%matplotlib inline
當月營收 = data.get('當月營收', 6) * 1000
當季營收 = 當月營收.iloc[-6:].sum() # 取近4個月營收總和，當作一季的月營收（4也可以改變）

# 將每季累計的財務數據，轉換成單季


# 計算自由現金流 = 營業活動之淨現金流入 - 投資活動之淨現金流出
投資現金流 = toSeasonal(data.get('投資活動之淨現金流入（流出）', 5))
營業現金流 = toSeasonal(data.get('營業活動之淨現金流入（流出）', 5))
自由現金流 = (投資現金流 + 營業現金流).iloc[-4:].sum()

稅後淨利 = data.get('本期淨利（淨損）', 5)
給自由現金的稅後淨利 = data.get('本期淨利（淨損）', 4)
# 修正：因為有些股東權益的名稱叫作「權益總計」有些叫作「權益總額」，所以要先將這兩個dataframe合併起來喔！
權益總計 = data.get('權益總計', 5)
權益總額 = data.get('權益總額', 5)

# 把它們合併起來（將「權益總計」為NaN的部分填上「權益總額」）
權益總計.fillna(權益總額, inplace=True)

股東權益報酬率 = 稅後淨利.iloc[-1] / 權益總計.iloc[-1]
股東權益報酬率.describe()

上季股東權益報酬率 = 稅後淨利.iloc[-5] / 權益總計.iloc[-5]
上季股東權益報酬率.describe()

營業利益 = data.get('營業利益（損失）', 5)
營業利益成長率 = (營業利益.iloc[-1] / 營業利益.iloc[-5] - 1) * 100
營業利益成長率.describe()

# current
流動資產 = data.get('流動資產合計',5)
流動負債 = data.get('流動負債合計',5)
current_ratio = 流動資產/流動負債
# liability
long_liability = data.get('非流動負債合計',5)

#銷貨
銷貨收入 = data.get('銷貨收入',6)
#capital
stocks = data.get('普通股股本',5)
#inventory
control_season = 2
inventory = data.get('存貨合計',6)
asset = data.get('資產總計',6)
no_turnover = inventory/asset
turnover_ratio = 1 - no_turnover
turnover_growth_rate = (turnover_ratio.iloc[-control_season] /turnover_ratio.iloc[-control_season-4])
turnover_growth_rate[0]

資產週轉率 = 銷貨收入/asset


# #負債比
# 負債 = data.get('負債總計',5)
負債比 = 負債 / asset
#營業利益率
營業收入 = data.get('營業收入合計',5)
營業利益率 = 營業利益/營業收入
# c1 = 當季營收 > 0
# #c2 = 自由現金流 >0
# c3 = (營業現金流.iloc[-1]>0)
# c4 = long_liability.iloc[-1] < long_liability.iloc[-5]
# c5 = current_ratio.iloc[-1] >current_ratio.iloc[-5]
# c6 = (stocks.iloc[-1] - stocks.iloc[-5]) <= 0
# c7 = 股東權益報酬率 > 上季股東權益報酬率
# c8 = 營業利益成長率>0 #把毛益率改成營業利益率
# c9 = turnover_growth_rate >0
# c10 = 股價淨值比 > 1
# c11 = 稅後淨利.iloc[-1] > 0

自由現金流 = 自由現金流/自由現金流.mean()

# 將條件做交集（&）
#select_stock = c1& c2 & c3& c4& c5& c6& c7& c8& c9 & c10 & c11

# 選出的股票
#select_stock = select_stock[select_stock]

In [252]:
拿出負債比 = 負債比.iloc[-1]/負債比.iloc[-5]
拿出現金流 = 自由現金流
拿出利益 = 營業利益/營業利益.mean()
拿出利益 =  拿出利益.iloc[-1]
yellowl_score = 拿出利益+ 拿出現金流 +拿出負債比

## Green score

In [195]:
#淨值比
流通股數= stocks / 10
每股淨值= 權益總計 / 流通股數
股價淨值比 = price.iloc[-1]/每股淨值.iloc[-1]  

#EPS
EPS = 稅後淨利/流通股數

#本益比
PE = price.iloc[-1] / EPS.iloc[-1]
PE = PE/10



累積營收 = data.get('累計營業收入合計',6)
累積營收
q1 = EPS.iloc[-1]/EPS.iloc[-2]
q2 = EPS.iloc[-2]/EPS.iloc[-3]
q3 = EPS.iloc[-3]/EPS.iloc[-4]
q4 = EPS.iloc[-4]/EPS.iloc[-5]
total_q = [q1,q2,q3,q4]
score_growth = gmean(total_q)-1
lss = pd.DataFrame(1 - score_growth)
lss = lss.T
stats_green  = EPS
stats_green = stats_green.append(lss).fillna(1)
temp = PE.dropna()
temp = pd.DataFrame(temp)
temp = temp.T
stats_green = stats_green.append(temp)
stats_green.fillna( stats_green.iloc[-1].mean()  ,inplace=True )
potential  = stats_green.iloc[-1] * stats_green.iloc[-2]
stats_green = stats_green.append(potential)
green_score = stats_green.iloc[-1]

/anaconda3/lib/python3.6/site-packages/scipy/stats/stats.py:305: RuntimeWarning: invalid value encountered in log
  log_a = np.log(np.array(a, dtype=dtype))
/anaconda3/lib/python3.6/site-packages/pandas/core/frame.py:6211: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


## Purple func

In [222]:
月增 = ((當月營收.iloc[-2] / 當月營收.iloc[-3]) -1)
年增 = ((當月營收.iloc[-2]/當月營收.iloc[-6])-1)
累積年增 = (累積營收.iloc[-1] /累積營收.iloc[-2] -1)
growth_01 = (1*年增 + 11 *月增 + 累積年增)/13
growth_02 = (1*年增 + 11 *月增 + 累積年增)/28 + growth_01
purple_score = growth_01 - growth_02

## Manipulate data

In [ ]:
green = -(green_score/green_score.mean())
yellow= -(yellowl_score/yellowl_score.mean())
purple= -(purple_score/purple_score.mean())

In [237]:
#g = pd.DataFrame(green)

In [241]:
def decide_range(colors):
    collect = []
    for color in colors:
        if color < 0.2:
            collect.append("優質")
        elif 0.6> color > 0.2:
            collect.append("安全")
        else:
            collect.append("危險")
    return collect

In [251]:
g = decide_range(green)
y = decide_range(yellow)
p = decide_range(purple)

In [278]:
len(yellow)

908

In [312]:
select = pd.concat([pd.DataFrame(y) , pd.DataFrame(g) , pd.DataFrame(p)],axis=1).dropna()
select.index = yellow.index
select.columns = ['yellow','green','purple']
select

,yellow,green,purple
stock_id,,,
1101,優質,優質,危險
1102,優質,優質,優質
1103,優質,優質,危險
1104,優質,優質,優質
1108,優質,危險,危險
1109,優質,優質,優質
1110,優質,優質,優質
1201,優質,優質,危險
1203,優質,優質,危險


In [311]:
# def lets_classify_stock(select):
#     for i in range(select.shape[0]):
#         if select.green[i] =="優質" and select.yellow[i]=="優質" and select.purple[i]=="優質":
#             select['kind'][i] = "優質股"
#         elif select.yellow[i] =="安全" and select.green[i]=="安全" and select.purple[i] =="優質":
#             select['kind'][i] = "小型飆股"
#         elif select.yellow[i] =="安全" and select.green[i]=="危險" and select.purple[i] =="危險":
#             select['kind'][i] = "高風險飆股"
#         elif select.yellow[i] =="危險" and select.green[i]=="危險" and select.purple[i] =="危險":
#             select['kind'][i] = "主力控盤高風險"
            
#         elif select.yellow[i] =="優質" and select.green[i]=="優質" and select.purple[i] =="安全":
#             select['kind'][i] = "主力控盤高風險"
#         elif select.yellow[i] =="危險" and select.green[i]=="安全" and select.purple[i] =="優質":
#             select['kind'][i] = "轉機股"
#         elif select.yellow[i] =="優質" and select.green[i]=="安全" and select.purple[i] =="危險":
#             select['kind'][i] = "轉壞股"
#         elif select.yellow[i] =="安全" and select.green[i]=="安全" and select.purple[i] =="安全":
#             select['kind'][i] = "定存股"
#         elif select.yellow[i] =="危險" and select.green[i]=="危險" and select.purple[i] =="安全":
#             select['kind'][i] = "奇蹟股" #兩好三壞

In [352]:
def lets_classify_stock(select):
    aselect = []
    for i in range(select.shape[0]):
        if select.green[i] =="優質" and select.yellow[i]=="優質" and select.purple[i]=="優質":
            aselect.append("優質股")
        elif select.yellow[i] =="優質" and select.green[i]=="優質" and select.purple[i] =="安全":
            aselect.append("主力控盤高風險")
        elif select.yellow[i] =="優質" and select.green[i]=="安全" and select.purple[i] =="危險":
            aselect.append("轉壞股")
        elif select.yellow[i] =="安全" and select.green[i]=="安全" and select.purple[i] =="優質":
            aselect.append("小型飆股")
        elif select.yellow[i] =="安全" and select.green[i]=="危險" and select.purple[i] =="危險":
            aselect.append("高風險飆股")
        elif select.yellow[i] =="安全" and select.green[i]=="安全" and select.purple[i] =="安全":
            aselect.append("定存股" )
        elif select.yellow[i] =="優質" and select.green[i]=="危險" and select.purple[i] =="危險":
            aselect.append("跌破股" )
        elif select.yellow[i] =="危險" and select.green[i]=="危險" and select.purple[i] =="危險":
            aselect.append("主力控盤高風險")
        elif select.yellow[i] =="危險" and select.green[i]=="優質" and select.purple[i] =="危險":
            aselect.append("反轉向下股")
        elif select.yellow[i] =="優質" and select.green[i]=="危險" and select.purple[i] =="優質":
            aselect.append("反轉向上股")
            
        elif select.yellow[i] =="危險" and select.green[i]=="安全" and select.purple[i] =="優質":
            aselect.append("轉機股")
      
        elif select.yellow[i] =="優質" and select.green[i]=="優質" and select.purple[i] =="危險":
            aselect.append("進場股")
        elif select.yellow[i] =="危險" and select.green[i]=="危險" and select.purple[i] =="安全":
            aselect.append("奇蹟股") #兩好三壞
        elif select.yellow[i] =="優質" and select.green[i]=="安全" and select.purple[i] =="優質":
             aselect.append("大型飆股")
        else:
             aselect.append('普通股')
    return aselect

In [359]:
mark_stock = lets_classify_stock(select)
mark_stock = pd.DataFrame(mark_stock,index=select.index)
mark_stock.columns = ["股票種類"]

,0
stock_id,
1101,進場股
1102,優質股
1103,進場股
1104,優質股
1108,跌破股
1109,優質股
1110,優質股
1201,進場股
1203,進場股


In [371]:
def get_stock_list(mark_stock):
    we = []
    for i in range(mark_stock.shape[0]):
        if mark_stock.股票種類[i]=="進場股" or mark_stock.股票種類[i]=="優質股" or mark_stock.股票種類[i]=="小型飆股":
            we.append(mark_stock.index[i])
    return we

In [373]:
lis = get_stock_list(mark_stock)
len(lis)

540